# HW09. Именованные сущности

## 1. Напишем правила на yargy, которые будут извлекать названия и характеристики автомобилей

In [1]:
from yargy import Parser, rule, or_, and_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display
from yargy.predicates import gram, is_capitalized, dictionary
from yargy.predicates import caseless, normalized, dictionary
from yargy.predicates import gte, lte
from razdel import sentenize
import pandas as pd

In [2]:
with open ('avito_cars.txt', 'r') as f: texts = f.read()

In [35]:
texts[25000:27000]

' диски менялись. Все документы из Автосервиса по ТО в наличии.Состояние салона отличное .Есть не большие потертости на кузове.Двигатель в идеальном состоянии .Масло менял каждые 8000-9000 тыс.км.Менялось масло в АККМ следующая замена только через тыс. 40000.\nв хорошем состоянии , документы все , номера в подарок....\nПродаю Срочно!!! Все вопросы при осмотре. Торг\nВ отличном состоянии. Установлен Вебасто, фаркоп.\nПродам хороший автомобиль в отличном состоянии. Я второй собственник отвечу на любые вопросы, автомобиль готов на все проверки за вас счёт. Максимальная комплектация, дополнительно установлен Фаркоп, ксенон в противотуманный фары, тонирование, сигнализация с автозапуском и турботаймером, Вебасто и многое другое. Все Техническое обслуживание у официального дилера митсубисши, отметки в сервисной книжке имеются! Автомобиль на новой зимней резине Nokian Hakkapeliitta 5, не вредному покупателю отдам с автомобилем комплект летних шин так же в отличном состоянии.\nОтличный рабочий

In [4]:
Car = fact(
    'Car',
    ['brand', 'model', 'year', 'mileage', 'price']
)

In [5]:
YEAR = and_(gte(1980), lte(2021))

YEAR_KEY = or_(
    rule('г', '.'),
    rule(normalized('год')),
    rule('г', '.', 'в', '.'),
    rule('г', '.', 'выпуска'),
    rule(normalized('год'), normalized('выпуск')),
    rule(normalized('год'), normalized('выпуск'), '–'),
    rule(normalized('год'), normalized('выпуск'), '-'),
    rule('г', '.', normalized('выпуск'), '-'),
    rule('г', '.', normalized('выпуск'), '–')
).named('KEY')
year = or_(
    rule
    (
    YEAR,
    YEAR_KEY.optional()
    ),
    rule
    (
    YEAR_KEY,
    YEAR
    )
).named('year')

In [6]:
MILEAGE = rule(and_(gte(10), lte(1000000)))

MILEAGE_KEY = or_(
    rule(normalized('Пробег')),
    rule(normalized('пробег')),
    rule(normalized('Пробег'), '-'),
    rule(normalized('Пробег'), '—'),
    rule(normalized('пробег'), '-'),
    rule(normalized('пробег'), '—'),
    rule(normalized('ПРОБЕГ'), '—'),
    rule(normalized('ПРОБЕГ'), '—'),
    rule('км', '.'),
    rule('км')
).named('KEY')
mileage = or_(
    rule
    (
    MILEAGE_KEY,
    MILEAGE
    ),
    rule
    (
    MILEAGE,
    MILEAGE_KEY
    )
).named('mileage')

In [7]:
PRICE = rule(and_(gte(10), lte(1000000)))

PRICE_KEY = or_(
    rule(normalized('Цена')),
    rule(normalized('цена')),
    rule(normalized('ЦЕНА')),
    rule(normalized('Цена'), '-'),
    rule(normalized('цена'), '-'),
    rule(normalized('ЦЕНА'), '-'),
    rule(normalized('Стоимость')),
    rule(normalized('стоимость')),
    rule(normalized('СТОИМОСТЬ')),
    rule(normalized('Стоимость'), '-'),
    rule(normalized('стоимость'), '-'),
    rule(normalized('СТОИМОСТЬ'), '-'),
    rule(normalized('рубли')),
    rule(normalized('Рубли')),
    rule(normalized('РУБЛИ')),
    rule('Рубл'),
    rule('рубл'),
    rule('РУБЛ'),
    rule('Руб'),
    rule('руб'),
    rule('РУБ'),
    rule('р', '.'),
    rule('Рубл', '.'),
    rule('рубл', '.'),
    rule('РУБЛ', '.'),
    rule('Руб', '.'),
    rule('руб', '.'),
    rule('РУБ', '.'),
    rule('тыс', '.'),
    rule('тыс'),
    rule('к', '.'),
    rule('тыщ', '.'),
    rule('тыщ'),
    rule('000')
).named('KEY')
price = or_(
    rule
    (
    PRICE_KEY,
    PRICE
    ),
    rule
    (
    PRICE,
    PRICE_KEY
    )
).named('price')

In [8]:
vaz = rule(
    morph_pipeline(['УАЗ', 'Уаз', 'уаз', 'Ваз', 'ВАЗ', 'ваз', 'vas', 'vaz', 'VAS', 'VAZ', 'vaz', 'Vaz']).interpretation(Car.brand),
    morph_pipeline(['21074', '2121', '2131', '2101', '2103', '2106', '21099', '1111']).interpretation(Car.model.normalized()),
    year.interpretation(Car.year.normalized()).optional(),
    mileage.interpretation(Car.mileage.normalized()).optional(),
    price.interpretation(Car.price.normalized()).optional()
    )

volvo = rule(
    morph_pipeline(['Вольво', 'ВОЛЬВО', 'вольво', 'VOLVO', 'volvo', 'Volvo']).interpretation(Car.brand),
    morph_pipeline(['ХС 70', 'XC70', 'xc70', 'xc 70', 'S40', 's40', 'S 40', 's 40', 'хс 90', 'XC90', 'XC 90', 'xc90', 'с60', 'C60', 'c 60', 'C 60', '850', 'S80', 'S 80', 's 80', 's80', '740', '760', '850', 'V90', 'V 90', 'v90', 'v 90', 'xc40', 'XC40', 'xc 40', 'XC 40', 'xc60', 'XC60', 'xc 60', 'XC 60']).interpretation(Car.model.normalized()),
    year.interpretation(Car.year.normalized()).optional(),
    mileage.interpretation(Car.mileage.normalized()).optional(),
    price.interpretation(Car.price.normalized()).optional()
    )

In [9]:
Car = or_(vaz, volvo).interpretation(Car)

parser = Parser(Car)

In [10]:
%%time
matches = list(parser.findall(texts))

CPU times: user 1min 54s, sys: 1.7 s, total: 1min 56s
Wall time: 1min 56s


In [11]:
all_cars = []
for m in matches:
    all_cars.append(m.fact)

In [12]:
print(len(all_cars))

535


In [13]:
all_cars = set(all_cars)

In [14]:
print(len(all_cars))

107


In [15]:
for car in all_cars:
    print(car.brand, car.model, car.year, car.mileage, car.price)

ВАЗ 1111 None None None
Ваз 2106 None None None
Ваз 2106 1990г. None None
ВАЗ 2101 None None None
Volvo XC90 2008г. None None
Вольво 850 None None None
Volvo S80 None None None
ваз 21099 год выпуск 1997 None None
ВАЗ 2106 None None цена 40000
Ваз 21074 2007 г.в. None None
ВАЗ 21074 2007 None None
VOLVO XC90 2004г. 141500 км None
ВАЗ 2131 2013 None None
ВАЗ 21099 2000 г.в. None None
ВАЗ 21099 1996 г.в. None None
Вольво S40 None None None
ВАЗ 21099 1999 год None None
ВАЗ 21099 None None None
ВАЗ 21074 2005 None None
Ваз 2121 None None None
ваз 21074 год выпуск 2011 None None
ВАЗ 21074 2005 год выпуск None None
ваз 2106 1994 год выпуск None None
ваз 21074 2007 год None None
ваз 21074 2008г. None None
Ваз 21074 2008 г. None None
Вольво S80 None None None
ваз 2106 1997 None None
VOLVO ХС 70 None None None
ВАЗ 2121 None None None
ВАЗ 2106 2001г.в. None None
ваз 2106 None None None
ваз 2106 1994г.в. None None
ВАЗ 21099 2002 г.в. None None
ВАЗ 21099 2001 год выпуск None None
ваз 21099 None Non

Бренд и модель парсит хорошо, а вот всё остальное -- не очень